In [1]:
import pandas as pd
from tqdm import tqdm
import os
import requests
import zipfile
from DeepPurpose import oneliner
from DeepPurpose.dataset import *

## Ligand conversion

In [2]:
DATA_FOLDER = "./molecules/"
url = 'https://storage.googleapis.com/indaba-challenge/molecules.zip'
r = requests.get(url, allow_redirects=True)
open('molecules.zip', 'wb').write(r.content)

# # Extract all the files with 
with zipfile.ZipFile('molecules.zip', 'r') as zip_ref:
    zip_ref.extractall(DATA_FOLDER)

In [3]:
extension = ".params"

In [4]:
import os
lines = None
smiles = 'SMILES'
DATA_FOLDER = "molecules/"
data = []
list_ligand_issues = []
start = -20
name = None
for filename in tqdm(os.listdir(DATA_FOLDER)):
    if filename.endswith(extension):
        filename = ".".join(filename.split('.')[:-1])+'.sdf'
        try:
            lines = open('{}{}'.format(DATA_FOLDER, filename)).read().split('\n')
        except:
            continue
        for num, line in enumerate(lines[start:]):
            if 'NAME' in line:
                name = lines[len(lines) + num + start + 1]
                continue
            if smiles in line:
                temp_smile = lines[len(lines) + num + start + 1]
                break
        if name is None:
            name = ".".join(filename.split('.')[:-1])
            
        name = "_".join(name.split()) + "_" + ".".join(filename.split('.')[:-1])
        if temp_smile is not None:
            data.append((name, temp_smile, filename))
        else:
            list_ligand_issues.append(DATA_FOLDER + filename)
        temp_smile = None
        name = None

100%|██████████| 12095/12095 [00:00<00:00, 34953.04it/s]


In [5]:
data = pd.DataFrame(data=data, columns=['Name', 'Smiles', 'Filename'])

In [6]:
data.head()

,Name,Smiles,Filename
0,medroxalol_MPQWSYJGFLADEW-PXAZEXFGSA-N,CC(CCc1ccc2c(c1)OCO2)NCC(O)c1ccc(O)c(C(N)=O)c1,MPQWSYJGFLADEW-PXAZEXFGSA-N.sdf
1,leucine_ROHFNLRQFUQHCH-YFKPBYRVSA-N,CC(C)C[C@H](N)C(=O)O,ROHFNLRQFUQHCH-YFKPBYRVSA-N.sdf
2,nortriptyline_PHVGLTMQBUFIQQ-UHFFFAOYSA-N,CNCCC=C1c2ccccc2CCc2ccccc21,PHVGLTMQBUFIQQ-UHFFFAOYSA-N.sdf
3,thiamazole_PMRYVIKBURPHAH-UHFFFAOYSA-N,Cn1cc[nH]c1=S,PMRYVIKBURPHAH-UHFFFAOYSA-N.sdf
4,salicylamide_SKZKKFZAGNVIMN-UHFFFAOYSA-N,NC(=O)c1ccccc1O,SKZKKFZAGNVIMN-UHFFFAOYSA-N.sdf


In [7]:
data.to_csv('Ligand_smiles.csv', index=False)

In [8]:
len(data)

4020

# Sample Drugs and Targets

In [10]:
targets = pd.read_csv('Leishmania_Target_Reduce_1.csv')[:10]
ligands = pd.read_csv('Ligand_smiles.csv')

In [11]:
ligands.head()

,Name,Smiles,Filename
0,medroxalol_MPQWSYJGFLADEW-PXAZEXFGSA-N,CC(CCc1ccc2c(c1)OCO2)NCC(O)c1ccc(O)c(C(N)=O)c1,MPQWSYJGFLADEW-PXAZEXFGSA-N.sdf
1,leucine_ROHFNLRQFUQHCH-YFKPBYRVSA-N,CC(C)C[C@H](N)C(=O)O,ROHFNLRQFUQHCH-YFKPBYRVSA-N.sdf
2,nortriptyline_PHVGLTMQBUFIQQ-UHFFFAOYSA-N,CNCCC=C1c2ccccc2CCc2ccccc21,PHVGLTMQBUFIQQ-UHFFFAOYSA-N.sdf
3,thiamazole_PMRYVIKBURPHAH-UHFFFAOYSA-N,Cn1cc[nH]c1=S,PMRYVIKBURPHAH-UHFFFAOYSA-N.sdf
4,salicylamide_SKZKKFZAGNVIMN-UHFFFAOYSA-N,NC(=O)c1ccccc1O,SKZKKFZAGNVIMN-UHFFFAOYSA-N.sdf


In [12]:
targets.head()

,Target_Name,Sequence
0,A0A640KCC9.0.apo,YPIINFTTAGATVQSYTNFIRAVRGRLTTGADVRHEIPVLPNRVGL...
1,E9BAY0.0.apo,MSRLMPHYSKGKTAFLCVDLQEAFSKRIENFANCVFVANRLARLHE...
2,O44010.0.apo,QYPIINFTTAGATVQSYTNFIRAVRGRLTTGADVRHEIPVLPNRVG...
3,Q4Q4I0.0.apo,SGRENLYFQGMTETFAFQAEINQLMSLIINTFYSNKEIFLRELISN...
4,Q25318.0.apo,VPDAVDWREKGAVTPVKDQGACGSCWAFSAVGNIEGQWYLAGHELV...


# Drug Selection

In [15]:
def formatLigand(ligands, filename):
    with open(filename, 'w') as f:
        for line in ligands.iterrows():
            f.write('{} {}\n'.format(line[1]['Name'], line[1]['Smiles']))

def formatTarget(target, filename):
    with open(filename, 'w') as f:
        for line in target.iterrows():
            f.write('{} {}\n'.format(line[1]['Target_Name'].split()[0], line[1]['Sequence']))

def BatchDrugRepurposing(ligands, target, step, filenameLigands='repurpose.txt', filenameTarget='target.txt'):
    formatLigand(ligands, filenameLigands)
    formatTarget(target, filenameTarget)
    targetName = target.iloc[0]['Target_Name']
    oneliner.repurpose(*read_file_target_sequence(filenameTarget), *read_file_repurposing_library(filenameLigands))
    outputFilenameRepurposing ="{}_{}".format(targetName, step)
    os.rename(r'./save_folder/results_aggregation/repurposing.txt', r'./save_folder/results_aggregation/' + targetName + str(step) + '.txt')
    
def DrurgRepurposing(ligands, target, batchSize = 100):
    ligandsSize = len(ligands)
    nStep = ligandsSize // batchSize
    remain = ligandsSize % batchSize != 0
    for step in tqdm(range(nStep), position=0, leave=True):
        start, end = step * batchSize, (step + 1) * batchSize
        batchLigands = ligands[start:end]
        BatchDrugRepurposing(batchLigands, target, step)
    
    if remain:
        start, end = nStep * batchSize, ligandsSize
        batchLigands = ligands[start:end]
        BatchDrugRepurposing(batchLigands, target, nStep)

In [16]:
listTargets = targets['Target_Name'].tolist()

In [ ]:
%%time
for target in tqdm(listTargets, position=0, leave=True):
    target = targets[targets['Target_Name'] == target]
    DrurgRepurposing(ligands, target)

  0%|          | 0/40 [00:00<?, ?it/s]

Save path not found or given and set to default: './save_folder/'. 
Loading customized repurposing dataset...
Beginning Downloading Pretrained Model...
Note: if you have already download the pretrained model before, please stop the program and set the input parameter 'pretrained_dir' to the path
Pretrained Models Successfully Downloaded...
Using pretrained model and making predictions...
repurposing...
in total: 100 drug-target pairs
encoding drug...
unique drugs: 100
drug encoding finished...
encoding protein...
unique target sequence: 1
protein encoding finished...
Done.
predicting...
---------------
Predictions from model 1 with drug encoding MPNN and target encoding CNN are done...
-------------
repurposing...
in total: 100 drug-target pairs
encoding drug...
unique drugs: 100
drug encoding finished...
encoding protein...
unique target sequence: 1
protein encoding finished...
Done.
predicting...
---------------
Predictions from model 2 with drug encoding CNN and target encoding CNN 

  2%|▎         | 1/40 [00:29<19:01, 29.28s/it]

models prediction finished...
aggregating results...
---------------
Drug Repurposing Result for A0A640KCC9.0.apo
+------+--------------------------------------------------------+------------------+---------------+
| Rank |                       Drug Name                        |   Target Name    | Binding Score |
+------+--------------------------------------------------------+------------------+---------------+
|  1   | beclometasone_dipropionate_KUVIULQEHSCUHY-XYWKZLDCSA-N | A0A640KCC9.0.apo |     25.87     |
|  2   |         naltrexone_DQCKKXVULJGBQN-XFWGSAIBSA-N         | A0A640KCC9.0.apo |     167.18    |
|  3   |         hycanthone_MFZWMTSUNYWVBU-UHFFFAOYSA-N         | A0A640KCC9.0.apo |     193.66    |
|  4   |       benzomethamine_XCEPXSCPQIRLCL-UHFFFAOYSA-N       | A0A640KCC9.0.apo |     199.70    |
|  5   |         pinaverium_DDHUTBKXLWCZCO-CEMLEFRQSA-N         | A0A640KCC9.0.apo |     212.54    |
|  6   |         cinalukast_BZMKNPGKXJAIDV-VAWYXSNFSA-N         | A0A640KCC9.0

  5%|▌         | 2/40 [00:58<18:33, 29.30s/it]

...

---------------
Predictions from model 5 with drug encoding Daylight and target encoding AAC are done...
-------------
models prediction finished...
aggregating results...
---------------
Drug Repurposing Result for A0A640KCC9.0.apo
+------+----------------------------------------------------------+------------------+---------------+
| Rank |                        Drug Name                         |   Target Name    | Binding Score |
+------+----------------------------------------------------------+------------------+---------------+
|  1   |        betamethasone_UREBDLICKHMUKA-DVTGEIKXSA-N         | A0A640KCC9.0.apo |      5.93     |
|  2   |       fluprednisolone_MYYIMZRZXIQBGI-HVIRSNARSA-N        | A0A640KCC9.0.apo |     28.55     |
|  3   |   etiprednol_dicloacetate_QAIOVDNCIZSSSF-RFAJLIJZSA-N    | A0A640KCC9.0.apo |     29.54     |
|  4   |        ethylestrenol_AOXRBFRFYPMWLR-XGXHKTLJSA-N         | A0A640KCC9.0.apo |     36.32     |
|  5   |    norethindrone_acetate_IMONTRJLAWHY

 32%|███▎      | 13/40 [07:17<15:23, 34.21s/it]

Predictions from model 5 with drug encoding Daylight and target encoding AAC are done...
-------------
models prediction finished...
aggregating results...
---------------
Drug Repurposing Result for A0A640KCC9.0.apo
+------+-------------------------------------------------------+------------------+---------------+
| Rank |                       Drug Name                       |   Target Name    | Binding Score |
+------+-------------------------------------------------------+------------------+---------------+
|  1   |   betamethasone_valerate_SNHRLVCMMWUAJD-SUYDQAKGSA-N  | A0A640KCC9.0.apo |     15.02     |
|  2   |        griseofulvin_DDUHZTYCFQRHIY-RBHXEPJQSA-N       | A0A640KCC9.0.apo |     51.85     |
|  3   |  fluorometholone_acetate_YRFXGQHBPBMFHW-SBTZIJSASA-N  | A0A640KCC9.0.apo |     80.98     |
|  4   |         rubitecan_VHXNKPBCCMUMSW-FQEVSTJZSA-N         | A0A640KCC9.0.apo |     190.30    |
|  5   |          estriol_PROQIPRRNZUXQM-ZXXIGWHRSA-N          | A0A640KCC9.0.apo |

 35%|███▌      | 14/40 [07:48<14:30, 33.48s/it]

---------------
Predictions from model 5 with drug encoding Daylight and target encoding AAC are done...
-------------
models prediction finished...
aggregating results...
---------------
Drug Repurposing Result for A0A640KCC9.0.apo
+------+----------------------------------------------------------+------------------+---------------+
| Rank |                        Drug Name                         |   Target Name    | Binding Score |
+------+----------------------------------------------------------+------------------+---------------+
|  1   |        triamcinolone_GFNANZIMVAIWHM-OBYCQNJPSA-N         | A0A640KCC9.0.apo |      8.07     |
|  2   |    fluocinolone_acetonide_FEBLZLNTKCEFIT-VSXGLTOVSA-N    | A0A640KCC9.0.apo |      9.88     |
|  3   |          moxestrol_MTMZZIPTQITGCY-OLGWUGKESA-N           | A0A640KCC9.0.apo |     20.61     |
|  4   |    isoflupredone_acetate_ZOCUOMKMBMEYQV-GSLJADNHSA-N     | A0A640KCC9.0.apo |     37.88     |
|  5   |         oxymesterone_RXXBBHGCAXVBES-X

## Data Merging

In [31]:
rootPath = './save_folder/results_aggregation/'

In [33]:
def readFile(rootPath, filename):
    data = []
    with open(rootPath+filename) as file:
        lines = file.readlines()[3:-1]
        for line in lines:
            line = line.replace(' ', '')
            items = line.split('|')
            data.append([items[2], items[3], items[4], None])
    
    return data

def mergeFile(targetName, rootPath, original=True):
    data = []
    for filename in tqdm(os.listdir(rootPath), position=0, leave=True):
        if targetName in filename:
            data.extend(readFile(rootPath, filename))
    data = pd.DataFrame(data=data, columns=['LigandName', 'TargetName', 'BindingScore', 'InterfE'])
    data = data.sort_values(by = ['BindingScore'])
    data.to_csv("Ligand_{}.csv".format(targetName), index=False)
    return data

In [34]:
for target in tqdm(listTargets, position=0, leave=True):
    mergeFile(target, rootPath)

100%|██████████| 13/13 [00:00<00:00, 18.43it/s]


In [48]:
for file in os.listdir('.'):
    if file.endswith('lig.csv') or file.endswith('apo.csv'):
        data = pd.read_csv(file).sort_values(by=['BindingScore'])
        data.to_csv(file, index=False)

In [49]:
result = pd.read_csv('Ligand_D0VWU6.0.apo.csv')

In [50]:
result.head()

,LigandName,TargetName,BindingScore,InterfE
0,mometasone_furoate_WOFMFGQZHJDGCX-ZULDAHANSA-N,D0VWU6.0.apo,4.29,NaN
1,ulobetasol_propionate_BDSYKGHYMJNPAB-LICBFIPMSA-N,D0VWU6.0.apo,7.76,NaN
2,clobetasone_butyrate_FBRAWBYQGRLCEK-AVVSTMBFSA-N,D0VWU6.0.apo,8.15,NaN
3,clobetasol_propionate_CBGUOGMQLZIXBE-XGQKBEPLSA-N,D0VWU6.0.apo,8.16,NaN
4,flumetasone_WXURHACBFYSXBI-GQKYHHCASA-N,D0VWU6.0.apo,10.53,NaN


In [39]:
result[-1:]

,LigandName,TargetName,BindingScore,InterfE
1135,glucose_WQZGKKKJIJFFOK-VFUOTHLCSA-N,D0VWU6.0.apo,1829498.66,NaN
